In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
from enbios2.base.experiment import Experiment
import bw2data

from enbios2.bw2.util import report
from enbios2.models.experiment_models import ExperimentData

In [3]:
# get an overview of brighway projects and databases
report()

Project: default
['biosphere3']
Project: ecoinvent
['biosphere3', 'cutoff_3.9.1_default']


In [4]:
bw2data.projects.set_current("ecoinvent")
database_name = 'cutoff_3.9.1_default'
db = bw2data.Database(database_name)

# Simple example experiment
In this first simple example we calculate the impact of 4 wind turbine activities in spain, using 2 different methods.

In [5]:
wind_turbines_spain = db.search("electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"})
wind_turbines_spain

Excluding 319 filtered results


['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, ES, None),
 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, ES, None)]

In [6]:
# for the experiment we need to create a list of activities (or a dict, where the keys represent the aliases)
# We need to add the codes, otherwise the brightway search will not be not uniquely identify the activities 
# adding name is just for convenience
experiment_activities = []

for activity in wind_turbines_spain:
    experiment_activities.append(
        {"id":
            {
                "name": activity["name"],
                "code": activity["code"]
            }
        }
    )

In [7]:
# we can modify the output of the activities, by default it is the reference product (1 of the activity unit)
experiment_activities[0]["output"] = ["kilowatt_hour", 3]
experiment_activities

[{'id': {'name': 'electricity production, wind, >3MW turbine, onshore',
   'code': '0d48975a3766c13e68cedeb6c24f6f74'},
  'output': ['kilowatt_hour', 3]},
 {'id': {'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'code': 'ed3da88fc23311ee183e9ffd376de89b'}},
 {'id': {'name': 'electricity production, wind, 1-3MW turbine, offshore',
   'code': '6ebfe52dc3ef5b4d35bb603b03559023'}},
 {'id': {'name': 'electricity production, wind, <1MW turbine, onshore',
   'code': '72cc067e1f4093c2e4c6ac9bdc93d844'}}]

In [8]:
# select 2 random methods and convert them into the form for enbios2
methods = [bw2data.methods.random() for _ in range(2)]
experiment_methods = [
    {
        "id": method
    }
    for method in methods
]

experiment_methods

[{'id': ('EDIP 2003 no LT', 'ecotoxicity no LT', 'acute, in water no LT')},
 {'id': ('Ecological Scarcity 2021 no LT',
   'water use no LT',
   'water resources, evaporated no LT')}]

In [9]:
# let's store the raw data, because we want to modify it later
raw_data = {
    "bw_project": "ecoinvent",
    "activities": experiment_activities,
    "methods": experiment_methods
}

# make a first validation of the experiment data
exp_data = ExperimentData(**raw_data)

In [10]:
# create a experiment object. This will validate the activities, their outputs, the methods and the scenarios.
exp: Experiment = Experiment(exp_data)

In [11]:
print(exp.info())

Experiment: 
Activities: 4
  electricity production, wind, >3MW turbine, onshore - electricity production, wind, >3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, onshore - electricity production, wind, 1-3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, offshore - electricity production, wind, 1-3MW turbine, offshore
  electricity production, wind, <1MW turbine, onshore - electricity production, wind, <1MW turbine, onshore
Methods: 2
 ('EDIP 2003 no LT', 'ecotoxicity no LT', 'acute, in water no LT')
 ('Ecological Scarcity 2021 no LT', 'water use no LT', 'water resources, evaporated no LT')
Hierarchy (depth): 2
Scenarios: 1


## Running the experiment

In [12]:
# run all scenarios at once
results = exp.run()

2023-08-03 15:43:58,146 - bw2calc - INFO - Initialized LCA object. Demand: {25415: 1, 15150: 1, 25751: 1, 25926: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x7f8da5ebcb10>, <bw_processing.datapackage.Datapackage object at 0x7f8dacf57050>, <bw_processing.datapackage.Datapackage object at 0x7f8dacf56810>, <bw_processing.datapackage.Datapackage object at 0x7f8da5f29e10>, <bw_processing.datapackage.Datapackage object at 0x7f8da5f2bdd0>, <bw_processing.datapackage.Datapackage object at 0x7f8db05f0d10>]
2023-08-03 15:43:58,147 - bw2calc - INFO - {'message': 'Started MultiLCA calculation', 'methods': [('EDIP 2003 no LT', 'ecotoxicity no LT', 'acute, in water no LT'), ('Ecological Scarcity 2021 no LT', 'water use no LT', 'water resources, evaporated no LT')], 'functional units': [[{'database': 'cutoff_3.9.1_default', 'code': '0d48975a3766c13e68cedeb6c24f6f74', 'amount': 3.0}], [{'database': 'cutoff_3.9.1_default', 'code': 'ed3da88fc23311ee183e9ffd376de89b', 'amount': 1.0}

# Result
The result is a dictionary of scenario names, where for each scenario we have a tree (representing the activity hierarchy). Each node (`BasicTreeNode`) in the tree has a `data` object, which is of the type `ScenarioResultNodeData`, which have the fields `output`, `result` and `bw_activity`.

In [13]:
# from enbios2.generic.tree.basic_tree import BasicTreeNode
# from enbios2.models.experiment_models import ScenarioResultNodeData

results

{'default scenario': [root - 4 children ]}

In [14]:
print(results["default scenario"].info())
print("---")
for children in results["default scenario"]:
    print(children.info())
    print("---")

[root - 4 children ]
ScenarioResultNodeData(output=('kilowatt_hour', 6.0), results={'EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT': 29.345704498534026, 'Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT': 1.1711947421892994}, bw_activity=None)
---
[electricity production, wind, >3MW turbine, onshore - 0 children  (root)]
ScenarioResultNodeData(output=('kilowatt_hour', 3.0), results={'EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT': 24.671384740510995, 'Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT': 0.6813872525495772}, bw_activity='electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None))
---
[electricity production, wind, 1-3MW turbine, onshore - 0 children  (root)]
ScenarioResultNodeData(output=('kilowatt_hour', 1.0), results={'EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT': 1.5230402743802678, 'Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporat

In [15]:
# we can dump the results into a csv file
exp.results_to_csv("test.csv")
pd.read_csv("test.csv").fillna('')

,lvl_0,lvl_1,unit,amount,"EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT (m3 water)","Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT (UBP)"
0,root,,kilowatt_hour,6.0,29.345704,1.171195
1,,"electricity production, wind, >3MW turbine, on...",kilowatt_hour,3.0,24.671385,0.681387
2,,"electricity production, wind, 1-3MW turbine, o...",kilowatt_hour,1.0,1.523040,0.134982
3,,"electricity production, wind, 1-3MW turbine, o...",kilowatt_hour,1.0,1.951144,0.186142
4,,"electricity production, wind, <1MW turbine, on...",kilowatt_hour,1.0,1.200136,0.168683


In [16]:
exp.scenarios[0].result_to_dict()

{'alias': 'root',
 'output': {'unit': 'kilowatt_hour', 'amount': 6.0},
 'results': {'EDIP 2003 no LT_eutrophication no LT_separate N potential no LT': 0.00012899929675533388,
  'CML v4.8 2016 no LT_ozone depletion no LT_ozone layer depletion (ODP steady state) no LT': 2.202663479917744e-09},
 'children': [{'alias': 'electricity production, wind, >3MW turbine, onshore',
   'output': {'unit': 'kilowatt_hour', 'amount': 3.0},
   'results': {'EDIP 2003 no LT_eutrophication no LT_separate N potential no LT': 8.622677360100662e-05,
    'CML v4.8 2016 no LT_ozone depletion no LT_ozone layer depletion (ODP steady state) no LT': 1.4346887570326957e-09},
   'bw_activity': '0d48975a3766c13e68cedeb6c24f6f74'},
  {'alias': 'electricity production, wind, 1-3MW turbine, onshore',
   'output': {'unit': 'kilowatt_hour', 'amount': 1.0},
   'results': {'EDIP 2003 no LT_eutrophication no LT_separate N potential no LT': 1.4156262380275074e-05,
    'CML v4.8 2016 no LT_ozone depletion no LT_ozone layer depl

## Add a technology hierarchy (dendrogram) 
Let's now add a few more activities to the experiment and create a hierarchy of activities.

In [16]:
solar_spain = db.search("solar", filter={"location": "ES"})[:2]
solar_spain

Excluding 465 filtered results


['electricity production, solar tower power plant, 20 MW' (kilowatt hour, ES, None),
 'electricity production, solar thermal parabolic trough, 50 MW' (kilowatt hour, ES, None)]

In [17]:
raw_data["activities"].extend([
    {
        "id": {
            "name": activity["name"],
            "code": activity["code"]
        }
    }
    for activity in solar_spain
])


In [18]:
hierarchy = {
    "wind": [wind_act["name"] for wind_act in wind_turbines_spain],
    "solar": [solar_act["name"] for solar_act in solar_spain]
}

raw_data["hierarchy"] = hierarchy
hierarchy

{'wind': ['electricity production, wind, >3MW turbine, onshore',
  'electricity production, wind, 1-3MW turbine, onshore',
  'electricity production, wind, 1-3MW turbine, offshore',
  'electricity production, wind, <1MW turbine, onshore'],
 'solar': ['electricity production, solar tower power plant, 20 MW',
  'electricity production, solar thermal parabolic trough, 50 MW']}

In [19]:
exp = Experiment(raw_data)
exp

Experiment: (call info() for details)
Activities: 6
Methods: 2
Hierarchy (depth): 3
Scenarios: 1

# Run the 2nd experiment

In [20]:
exp.run()

2023-08-03 15:46:28,969 - bw2calc - INFO - Initialized LCA object. Demand: {25415: 1, 15150: 1, 25751: 1, 25926: 1, 13409: 1, 11123: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x7f8dac7cdc10>, <bw_processing.datapackage.Datapackage object at 0x7f8dad287650>, <bw_processing.datapackage.Datapackage object at 0x7f8dad096850>, <bw_processing.datapackage.Datapackage object at 0x7f8dae303cd0>, <bw_processing.datapackage.Datapackage object at 0x7f8da5eb92d0>, <bw_processing.datapackage.Datapackage object at 0x7f8da5eb9610>, <bw_processing.datapackage.Datapackage object at 0x7f8da5e7e350>, <bw_processing.datapackage.Datapackage object at 0x7f8dad142750>]
2023-08-03 15:46:28,970 - bw2calc - INFO - {'message': 'Started MultiLCA calculation', 'methods': [('EDIP 2003 no LT', 'ecotoxicity no LT', 'acute, in water no LT'), ('Ecological Scarcity 2021 no LT', 'water use no LT', 'water resources, evaporated no LT')], 'functional units': [[{'database': 'cutoff_3.9.1_default', 'code

{'default scenario': [root - 2 children ]}

In [22]:
# print(json.dumps((exp.scenarios[0].result_to_dict()), indent=2))
exp.scenarios[0].results_to_csv("test.csv", level_names=["root", "technology", "activity"])
pd.read_csv("test.csv").fillna('')

,root,technology,activity,unit,amount,"EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT","Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT"
0,root,,,kilowatt_hour,8.0,33.019308,1.629857
1,,wind,,kilowatt_hour,6.0,29.345704,1.171195
2,,,"electricity production, wind, >3MW turbine, on...",kilowatt_hour,3.0,24.671385,0.681387
3,,,"electricity production, wind, 1-3MW turbine, o...",kilowatt_hour,1.0,1.523040,0.134982
4,,,"electricity production, wind, 1-3MW turbine, o...",kilowatt_hour,1.0,1.951144,0.186142
5,,,"electricity production, wind, <1MW turbine, on...",kilowatt_hour,1.0,1.200136,0.168683
6,,solar,,kilowatt_hour,2.0,3.673604,0.458662
7,,,"electricity production, solar tower power plan...",kilowatt_hour,1.0,1.277954,0.207914
8,,,"electricity production, solar thermal paraboli...",kilowatt_hour,1.0,2.395649,0.250748


## Create several scenarios

In [23]:
from random import randint

activitiy_aliases = exp.get_all_activity_aliases()

def create_random_scenario():
    return {
        "activities": {
            act: ["kilowatt_hour", randint(1, 10)]
            for act in activitiy_aliases
        }
    }

raw_data["scenarios"] = [
    create_random_scenario(),
    create_random_scenario()
]

raw_data["scenarios"]

[{'activities': {'electricity production, wind, >3MW turbine, onshore': ['kilowatt_hour',
    5],
   'electricity production, wind, 1-3MW turbine, onshore': ['kilowatt_hour',
    2],
   'electricity production, wind, 1-3MW turbine, offshore': ['kilowatt_hour',
    6],
   'electricity production, wind, <1MW turbine, onshore': ['kilowatt_hour',
    10],
   'electricity production, solar tower power plant, 20 MW': ['kilowatt_hour',
    7],
   'electricity production, solar thermal parabolic trough, 50 MW': ['kilowatt_hour',
    6]}},
 {'activities': {'electricity production, wind, >3MW turbine, onshore': ['kilowatt_hour',
    7],
   'electricity production, wind, 1-3MW turbine, onshore': ['kilowatt_hour',
    7],
   'electricity production, wind, 1-3MW turbine, offshore': ['kilowatt_hour',
    8],
   'electricity production, wind, <1MW turbine, onshore': ['kilowatt_hour', 4],
   'electricity production, solar tower power plant, 20 MW': ['kilowatt_hour',
    2],
   'electricity production,

In [24]:
exp = Experiment(raw_data)

ValueError: Activity ExperimentActivityId(database=None, code='ed3da88fc23311ee183e9ffd376de89b', name='electricity production, wind, 1-3MW turbine, onshore', location=None, unit=None, alias=None) has invalid output format: 2 validation errors for ExperimentActivityData
output
  instance of ActivityOutput, tuple or dict expected (type=type_error.dataclass; class_name=ActivityOutput)
output
  value is not a valid tuple (type=type_error.tuple)

## Run the experiment for the 3rd time
This time will likely take some more time since we need to run 2 scenarios. 

In [28]:
_ = exp.run()

2023-08-03 15:48:58,892 - bw2calc - INFO - Initialized LCA object. Demand: {25415: 1, 15150: 1, 25751: 1, 25926: 1, 13409: 1, 11123: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x7f8dacd05810>, <bw_processing.datapackage.Datapackage object at 0x7f8dacd075d0>, <bw_processing.datapackage.Datapackage object at 0x7f8e07288ed0>, <bw_processing.datapackage.Datapackage object at 0x7f8dad132590>, <bw_processing.datapackage.Datapackage object at 0x7f8dacc93e50>, <bw_processing.datapackage.Datapackage object at 0x7f8daccd8550>, <bw_processing.datapackage.Datapackage object at 0x7f8dacc9d890>, <bw_processing.datapackage.Datapackage object at 0x7f8dad135a50>]
2023-08-03 15:48:58,893 - bw2calc - INFO - {'message': 'Started MultiLCA calculation', 'methods': [('EDIP 2003 no LT', 'ecotoxicity no LT', 'acute, in water no LT'), ('Ecological Scarcity 2021 no LT', 'water use no LT', 'water resources, evaporated no LT')], 'functional units': [[{'database': 'cutoff_3.9.1_default', 'code

In [29]:
exp.scenarios[0].results_to_csv("s1.csv", level_names=["root", "technology", "activity"])
pd.read_csv("s1.csv").fillna('')

,root,technology,activity,unit,amount,"EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT","Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT"
0,root,,,kilowatt_hour,8.0,99.057924,4.889571
1,,wind,,kilowatt_hour,6.0,58.691409,2.342389
2,,,"electricity production, wind, >3MW turbine, on...",kilowatt_hour,3.0,24.671385,0.681387
3,,,"electricity production, wind, 1-3MW turbine, o...",kilowatt_hour,1.0,1.523040,0.134982
4,,,"electricity production, wind, 1-3MW turbine, o...",kilowatt_hour,1.0,1.951144,0.186142
5,,,"electricity production, wind, <1MW turbine, on...",kilowatt_hour,1.0,1.200136,0.168683
6,,solar,,kilowatt_hour,2.0,7.347207,0.917324
7,,,"electricity production, solar tower power plan...",kilowatt_hour,1.0,1.277954,0.207914
8,,,"electricity production, solar thermal paraboli...",kilowatt_hour,1.0,2.395649,0.250748


In [30]:
raw_data["scenarios"]

[{'activities': {'electricity production, wind, >3MW turbine, onshore': ['kilowatt_hour',
    5],
   'electricity production, wind, 1-3MW turbine, onshore': ['kilowatt_hour',
    2],
   'electricity production, wind, 1-3MW turbine, offshore': ['kilowatt_hour',
    6],
   'electricity production, wind, <1MW turbine, onshore': ['kilowatt_hour',
    10],
   'electricity production, solar tower power plant, 20 MW': ['kilowatt_hour',
    7],
   'electricity production, solar thermal parabolic trough, 50 MW': ['kilowatt_hour',
    6]}},
 {'activities': {'electricity production, wind, >3MW turbine, onshore': ['kilowatt_hour',
    7],
   'electricity production, wind, 1-3MW turbine, onshore': ['kilowatt_hour',
    7],
   'electricity production, wind, 1-3MW turbine, offshore': ['kilowatt_hour',
    8],
   'electricity production, wind, <1MW turbine, onshore': ['kilowatt_hour', 4],
   'electricity production, solar tower power plant, 20 MW': ['kilowatt_hour',
    2],
   'electricity production,

## Inspecting the results

We can now do some transformations of the results. For that is useful to know how to retrieve is singular result from a scenario result. 
The result of a scenario is a tree structure, where the nodes `name`s are activity aliases or names defined in the hierarchy. With the function of BasicTreeNode.find_child_by_name we can directly access the result of a node.    

Following we transform the results into a dictionary of the following structure:
```json
{
    "activity_alias": {
        "method_alias": "[list of results for each scenario]"
    }
}
```

In [31]:
all_results = {}
for activity in activitiy_aliases:
    all_results[activity] = {method_alias : [] for method_alias in exp.get_all_method_aliases()}
    for scenario in exp.scenarios:
        activity_result = scenario.result_tree.find_child_by_name(activity)
        for method, score in activity_result.data.results.items():
            all_results[activity][method].append(score)
            
all_results

{'electricity production, wind, >3MW turbine, onshore': {'EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT': [24.671384740510995],
  'Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT': [0.6813872525495772]},
 'electricity production, wind, 1-3MW turbine, onshore': {'EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT': [1.5230402743802678],
  'Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT': [0.13498243649342634]},
 'electricity production, wind, 1-3MW turbine, offshore': {'EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT': [1.9511435306496767],
  'Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT': [0.18614175911721723]},
 'electricity production, wind, <1MW turbine, onshore': {'EDIP 2003 no LT_ecotoxicity no LT_acute, in water no LT': [1.2001359529930862],
  'Ecological Scarcity 2021 no LT_water use no LT_water resources, evaporated no LT': [0.1686832940290786]},
 'elect